
A simple code to scrape information from the google earth engine. Currently, the google earth engine does not allow to search for a dataset containing bands of a specific wavelength and for a particular duration.

I considered only two parameters for the first version.

Specify the date range.

//Define the starting date.

startingdate = date(2021, 1, 1)

//Define the ending date.

endingdate = date(2021, 12, 31)

Specify the wavelength symbol.

if 'µ' in soup.getText() or 'μ' in soup.getText():


In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
from datetime import timezone
from datetime import datetime
from collections import namedtuple
Range = namedtuple('Range', ['start', 'end'])
from datetimerange import DateTimeRange
from datetime import date
MATCH_ALL = r'.*'

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def like(string):
    """
    Return a compiled regular expression that matches the given
    string with any prefix and postfix, e.g. if string = "hello",
    the returned regex matches r".*hello.*"
    """
    string_ = string
    if not isinstance(string_, str):
        string_ = str(string_)
    regex = MATCH_ALL + re.escape(string_) + MATCH_ALL
    return re.compile(regex, flags=re.DOTALL)

def find_by_text(soup, text, tag, **kwargs):
 
    elements = soup.find_all(tag, **kwargs)
    matches = []
    for element in elements:
        if element.find(text=like(text)):
            matches.append(element)
    if len(matches) > 1:
        raise ValueError("Too many matches:\n" + "\n".join(matches))
    elif len(matches) == 0:
        return None
    else:
        return matches[0] 


def with_in_date_range(soup,startingdate,endingdate):
    dates = find_by_text(soup,'Dataset Availability','dl').findAll('dd')[0].text.split('–')
    starts = datetime.fromisoformat(dates[0][0:10]).astimezone(timezone.utc).strftime('%Y-%m-%d').split(" ")[0].split("-")
    ends = datetime.fromisoformat(dates[1][0:10]).astimezone(timezone.utc).strftime('%Y-%m-%d').split(" ")[0].split("-")
    if date(int(starts[0]),int(starts[1]),int(starts[2]))<= startingdate and date(int(ends[0]),int(ends[1]),int(ends[2]))>= endingdate:
        return 1
    else:
        #print("Dataset is not available in the specified range.")
        return 0

def get_Text(url):
    reqs = requests.get(url)
    soup = BeautifulSoup(reqs.text, 'html.parser')
    return soup

# URL contains the catalog for all the datasets on the google earth engine.

url="https://developers.google.com/earth-engine/datasets/catalog"
# Define the starting date.
startingdate = date(2021, 1, 1)
# Define the ending date.
endingdate = date(2021, 12, 31)






urls = []
# Find the link to each dataset.
data = get_Text(url)
data = data.findAll('td',attrs={'class':'ee-dataset'})
count = 0

# Store the links.
finallinks = []
for div in data:
 datasets = div.findAll('a')
 for dataset in datasets:
  soup = get_Text("https://developers.google.com" + dataset['href'])
  
  # Check if the dataset on google earth engine is within the specified date range.

  if with_in_date_range(soup,startingdate,endingdate):
   # If the dataset is within range, search for the specific keywords in the documents.
   [s.extract() for s in soup(['style', 'script', '[document]', 'head', 'title'])]
   if 'µ' in soup.getText() or 'μ' in soup.getText():
    #if "cloud" in soup.getText() or "infrared" in soup.getText() or 'µ' in soup.getText():
    print("Link :",count, "https://developers.google.com" + dataset['href'])

    # Append the link to final links.
    finallinks.append("https://developers.google.com" + dataset['href'])

    count=count+1


print("Number of datasets: ",count)

# Save the links in file.
with open('finallinks.txt', 'w') as f:
    for line in finallinks:
        f.write(line)
        f.write('\n')

Code to download images from google earth engine.

In [1]:
from heapq import merge
from operator import index
import pandas as pd
import numpy as np
import os
import re
import pandas as pd
import numpy as np
import os
import re
import pandas as pd
import numpy as np
import sklearn.neighbors
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import time
from multiprocessing import Process
import ee
import pandas as pd
import ee
import ee
import geemap
import os
import rasterio
from rasterio.plot import show
import os
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show_hist
from rasterio.plot import show_hist


ee.Initialize()

# Helper function to convert the square into earth engine polygon.

def findpolygon_earthengine(row):
 return [[row['topleft_long'],row['topleft_lat']],
 [row['topright_long'],row['topright_lat']],
 [row['bottomright_long'],row['bottomright_lat']],
 [row['bottomleft_long'],row['bottomleft_lat']], 
 [row['topleft_long'],row['topleft_lat']]]

# Helper function to convert the square into geopandas polygon.
def findpolygon_geopandas(row):
  return Polygon([( row['topleft_long'],row['topleft_lat'])
    , ( row['topright_long'],row['topright_lat'])
    ,(row['bottomright_long'],row['bottomright_lat'] )
    , ( row['bottomleft_long'],row['bottomleft_lat'])
    , (row['topleft_long'],row['topleft_lat'] )])

def makeboxes(data):
  # This constant value is added to each longitude and latitude value to make a polygon.
  constant = 0.1
  data['topleft_lat'] = data['Latitude'] - constant
  data['topleft_long'] = data['Longitude'] -constant
  data['topright_lat'] = data['Latitude'] - constant
  data['topright_long'] = data['Longitude'] + constant
  data['bottomleft_lat'] = data['Latitude'] + constant
  data['bottomleft_long'] = data['Longitude'] - constant
  data['bottomright_lat'] = data['Latitude'] +constant
  data['bottomright_long'] = data['Longitude'] + constant
  data['p'] = data.apply(findpolygon_geopandas, axis=1)
  data['coords'] = data.apply(findpolygon_earthengine, axis=1)
  return data


def sorted_nicely( l ):
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key = alphanum_key)

location = pd.DataFrame()
location['Latitude'] = [24.3722]
location['Longitude'] = [54.467]
location['Location'] = ["Center"]
print(location)

location = makeboxes(location)

# Here, we defined a specific dataset containing cloud presence information.
# Define the google earth engine dataset and the corresponding band.
names = ['NOAA/NCEP_DOE_RE2/total_cloud_coverage']
bands = ['tcdc']
datasetnames = pd.DataFrame()
datasetnames['name'] = names
datasetnames['bands'] = bands

# Define the starting and the ending range.
i_date = '2021-01-01'
f_date =  '2021-01-02'

# Use this function to download earth engine images for each station separately.
def downloadindividualdataset():
 for index, row in location.iterrows():
  roi = ee.Geometry.Polygon(row['coords'])
  for index2,row2 in datasetnames.iterrows():
   # Read the earth engine dataset.
   collection = (ee.ImageCollection(row2['name']).select(row2['bands']).filterDate(i_date ,f_date).filterBounds(roi))
   
   # Crop the specific portion of an image.
   collection =  collection.map(lambda image: image.clip(roi))
   
   # Make a directory with a station name to store the images.  
   if not os.path.isdir(row['Location']): 
    os.mkdir(row['Location'])   
   directoryname = row['Location']+os.sep+"images" 
   if not os.path.isdir(directoryname): 
    os.mkdir(directoryname)
   print(collection.aggregate_array('system:index').getInfo())
   geemap.ee_export_image_collection(collection, out_dir=directoryname)
   geemap.ee_export_image_collection_to_drive(collection, folder='export', scale=30)

downloadindividualdataset()


   Latitude  Longitude Location
0   24.3722     54.467   Center
['tcdc_2021010100', 'tcdc_2021010106', 'tcdc_2021010112', 'tcdc_2021010118', 'tcdc_2021010200', 'tcdc_2021010206', 'tcdc_2021010212', 'tcdc_2021010218', 'tcdc_2021010300', 'tcdc_2021010306', 'tcdc_2021010312', 'tcdc_2021010318', 'tcdc_2021010400', 'tcdc_2021010406', 'tcdc_2021010412', 'tcdc_2021010418', 'tcdc_2021010500', 'tcdc_2021010506', 'tcdc_2021010512', 'tcdc_2021010518', 'tcdc_2021010600', 'tcdc_2021010606', 'tcdc_2021010612', 'tcdc_2021010618', 'tcdc_2021010700', 'tcdc_2021010706', 'tcdc_2021010712', 'tcdc_2021010718', 'tcdc_2021010800', 'tcdc_2021010806', 'tcdc_2021010812', 'tcdc_2021010818', 'tcdc_2021010900', 'tcdc_2021010906', 'tcdc_2021010912', 'tcdc_2021010918']
Total number of images: 36

Exporting 1/36: tcdc_2021010100.tif
Generating URL ...
Please wait ...
Data downloaded to /home/muneeb/Desktop/Paper2/Procedure 5/Center/images/tcdc_2021010100.tif


Exporting 2/36: tcdc_2021010106.tif
Generating URL ...
Pl


KeyboardInterrupt

